### Importing the Libraries

In [1]:
import pandas as pd
import numpy as np

In total, we have 55 files that need to be merged. We will do this in the next step.

In [2]:
#df = pd.read_csv("data/Kickstarter000.csv")

#for i in range(1,56):
    
  #  num = str(1000 + i)
    
  #  dfi = pd.read_csv(f"data/Kickstarter{num[1:4]}.csv")
  #  df = pd.concat([df,dfi], axis = 0)
    
#df.to_csv("data/kickstarter_raw.csv")

The new dataframe contains 209222 observations and 37 columns.

In [3]:
df = pd.read_csv("data/kickstarter_raw.csv")

### Data Preparation

At first we need to get a overview of the data to see where data cleaning is needed.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209222 entries, 0 to 209221
Data columns (total 38 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                209222 non-null  int64  
 1   backers_count             209222 non-null  int64  
 2   blurb                     209214 non-null  object 
 3   category                  209222 non-null  object 
 4   converted_pledged_amount  209222 non-null  int64  
 5   country                   209222 non-null  object 
 6   created_at                209222 non-null  int64  
 7   creator                   209222 non-null  object 
 8   currency                  209222 non-null  object 
 9   currency_symbol           209222 non-null  object 
 10  currency_trailing_code    209222 non-null  bool   
 11  current_currency          209222 non-null  object 
 12  deadline                  209222 non-null  int64  
 13  disable_communication     209222 non-null  b

Missing values
As we can see from above "blurb", "friends", "is_backing", "is_starred", "location", "permissions" and "usd_type" contain missing values. The variables "friends", "is_backing" ... contain missing values. We will clean them later.. bla ... Drop vars with more than 70% missing values?

usd_type modus?
location
blurb delete rows?

Variable types
datetime format and drop of old date columns




In [5]:
# convert unix date time to pandas datetime format
df['created_at'] = pd.to_datetime(df['created_at'],unit='s')
df['deadline'] = pd.to_datetime(df['deadline'],unit='s')
df['launched_at'] = pd.to_datetime(df['launched_at'],unit='s')

# pick out year, month and weekday out of date of launching
df['yr_launched'] = df['launched_at'].dt.year
df['mo_launched'] = df['launched_at'].dt.month
df['wd_launched'] = df['launched_at'].dt.weekday

# pick out year, month and weekday out of date of deadline
df['yr_deadline'] = df['deadline'].dt.year
df['mo_deadline'] = df['deadline'].dt.month
df['wd_deadline'] = df['deadline'].dt.weekday

In [6]:
# calculate duration of founding_phase (time between launching and deadline)
founding_phase = df['deadline'] - df['launched_at']
df['founding_phase'] = founding_phase.dt.days

# calculate duration of phase_before (time between creation and launcing)
phase_before = df['launched_at'] - df['created_at']
df['phase_before'] = phase_before.dt.days

In [7]:
# drop columns created_at, deadline, launched_at
df = df.drop(["created_at", "deadline", "launched_at"], axis=1)


In [8]:
# 2. Inspection object types... category; text; probably important info -> extraction of content

In [9]:
df["category_dict"] = df["category"].apply(eval)
df_category = df["category_dict"].apply(pd.Series)
df_category.head()

,id,name,slug,position,parent_id,color,urls
0,43,Rock,music/rock,17,14.0,10878931,{'web': {'discover': 'http://www.kickstarter.c...
1,54,Mixed Media,art/mixed media,6,1.0,16760235,{'web': {'discover': 'http://www.kickstarter.c...
2,280,Photobooks,photography/photobooks,5,15.0,58341,{'web': {'discover': 'http://www.kickstarter.c...
3,266,Footwear,fashion/footwear,5,9.0,16752598,{'web': {'discover': 'http://www.kickstarter.c...
4,51,Software,technology/software,11,16.0,6526716,{'web': {'discover': 'http://www.kickstarter.c...


In [10]:
print(df_category.shape)
all_categories = df_category[["id", "name"]].drop_duplicates("name")
all_categories.rename(columns = {"id": "parent_id"}, inplace = True)
all_categories.head()

(209222, 7)


,parent_id,name
0,43,Rock
1,54,Mixed Media
2,280,Photobooks
3,266,Footwear
4,51,Software


In [11]:
df_category = df_category.merge( all_categories, how = "left", on = "parent_id", suffixes = ["", "_parent"])
df["main_category"] = df_category["name_parent"]
df["subcategory"] = df_category["name"]

In [12]:
# Drop "category_dict" column
df = df.drop(["category", "category_dict"], axis=1)

In [13]:
# Extract name, state and type from location var...
# Rename name var into ?
# Drop location

df["location"].fillna("{}", inplace = True)

df["location"] = df["location"].str.replace(',"is_root":false', "")
df["location"] = df["location"].str.replace(',"is_root":true', "" )
df["location"] = df["location"].str.replace('null', "{}" )

df["location_dict"] = df["location"].apply(eval)
df_location = df["location_dict"].apply(pd.Series)
df_location.head()

df["city"] = df_location["name"]
df["region"] = df_location["state"]
df["city_type"] = df_location["type"]

In [14]:
# Drop "location" column
df = df.drop(["location", "location_dict"], axis=1)

### To-do Get creator id out of creator

In [15]:
#df["creator"] = df["creator"].str.replace('"is_registered":null, ', "" )
#df["creator"] = df["creator"].str.replace('"chosen_currency":null, ', "" )
#df["creator"] = df["creator"].str.replace('null', "'hi'")

#df["creator_dict"] = df["creator"].apply(eval)
#df_creator = df["creator_dict"].apply(pd.Series)

#df["user_id"] = df_creator["id"]
df.columns

Index(['Unnamed: 0', 'backers_count', 'blurb', 'converted_pledged_amount',
       'country', 'creator', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'disable_communication',
       'friends', 'fx_rate', 'goal', 'id', 'is_backing', 'is_starrable',
       'is_starred', 'name', 'permissions', 'photo', 'pledged', 'profile',
       'slug', 'source_url', 'spotlight', 'staff_pick', 'state',
       'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type', 'yr_launched', 'mo_launched', 'wd_launched', 'yr_deadline',
       'mo_deadline', 'wd_deadline', 'founding_phase', 'phase_before',
       'main_category', 'subcategory', 'city', 'region', 'city_type'],
      dtype='object')

In [16]:
# Exclusion of the following X columns
# Add above? 
df = df.drop(["currency_symbol",
         "currency_trailing_code",
         "friends",
         "is_backing",
         "is_starred",
         "permissions", 
         "photo",
         "source_url",
         "urls",
         "state_changed_at",
         "spotlight",
         "Unnamed: 0"], axis=1)


         #,"Unnamed: 0.1"

In [17]:
# Exclusion other levels than "successful and "failed". But at first, have a look at the counts...
# Reason why exclusion of other levels...

df.state.value_counts()

# successful    117465
# failed         75199

# 192664 Sum of "successful" and "failed"

df = df.query('state in ["successful", "failed"]')

# Prove if it works
df.state.value_counts()
df.shape

# After dropping rows and colums the data set contains X observations and Y columns


(192664, 34)

For our model/analysis we need to create a X variable of the target va

In [18]:
# Encoding of the state (target) variable:
df['state'] = df['state'].apply(lambda x: 1 if x == 'successful' else 0)

# 0 = failed
# 1 = sucessful

In [19]:
# Blurb var 
# Hypothesis
# Extract number of words
df['words_blurb'] = df['blurb'].apply(lambda x: len(str(x).split()))


# Sentiment analysis?

Spotlight allows creators to **make a home for their project on Kickstarter after they’ve been successfully funded.** Each creator can take control of their page and build a customized, central hub for news, updates, links to finished work, and anything else they want the world to know about their project.


In [20]:
#df[["static_usd_rate", "fx_rate", "currency", "current_currency", "usd_pledged", "pledged", "usd_type"]].head(20)
#test = df[["state", "currency", "usd_pledged", "pledged", "goal"]]

#test["compare"] = test["goal"] - test["pledged"]
#test[test["state"] == "successful"]
#test.query("state == 'successful' and compare > 0")

#df.currency.unique()


#61310 index --> exclusion (nicht stimmig)

<ipython-input-20-d675ab0df0b1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["compare"] = test["goal"] - test["pledged"]


,state,currency,usd_pledged,pledged,goal,compare


In [21]:
df["goal_usd"] = df["goal"]*df["static_usd_rate"]

In [22]:
df = df.drop(["creator",
             "blurb",
             "static_usd_rate",
             "fx_rate",
             "currency",
             "current_currency",
             "pledged",
             "goal",
             "profile",
             "converted_pledged_amount",
             "slug"], axis=1)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192664 entries, 0 to 209221
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   backers_count          192664 non-null  int64  
 1   country                192664 non-null  object 
 2   disable_communication  192664 non-null  bool   
 3   id                     192664 non-null  int64  
 4   is_starrable           192664 non-null  bool   
 5   name                   192664 non-null  object 
 6   staff_pick             192664 non-null  bool   
 7   state                  192664 non-null  int64  
 8   usd_pledged            192664 non-null  float64
 9   usd_type               192626 non-null  object 
 10  yr_launched            192664 non-null  int64  
 11  mo_launched            192664 non-null  int64  
 12  wd_launched            192664 non-null  int64  
 13  yr_deadline            192664 non-null  int64  
 14  mo_deadline            192664 non-nu

In [ ]:
# Ex id for model; analysis duplicates of id for bus. understand. (compare rows)

df.id.nunique()

Final vars am Ende beschreiben

In [81]:
# df.to_csv("data/data_clean.csv")
#df = pd.read_csv("data/data_clean.csv")